# **PROYECTO FINAL PYTHON**
---

Daniela Franco León


# Descargar Librerías necesarias

In [ ]:
!pip install kraken
!pip install krakenex
!pip install pykrakenapi 

In [4]:
#Para conectar con la plataforma Kraken y acceder a su información
import krakenex
from pykrakenapi import KrakenAPI
import requests
#Para la limpieza y estructuración general de los datos obtenidos
import pandas as pd
import datetime 
import pprint
#Para las visualizaciones creadas
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

# Lectura y representación del movimiento del par de monedas

## Carga de datos-librería de Kraken

### Implementación Inicial 



---

La información a continuación representa los pasos previos que se dieron para poder llegar al resultado final. En este caso, comencé cargando la información OHLC de cada moneda por separado, para así poder analizar el DataFrame que obtenía.

Esta no es la forma más práctica de cargar los datos moneda por moneda, pero es como inicialmente comencé a cargar la información y la base a cómo se creó el resultado final de este trabajo.

Lo dejo comentado ya que no me aporta nada más al resultado final del proyecto, pero fue parte del proceso. 

In [5]:
#Primero conectamos con Kraken API 
api = krakenex.API()
k = KrakenAPI(api)

#DESCARGAR PRIMERA MONEDA
INCHEUR, last = k.get_ohlc_data("1INCHEUR",interval=1440, ascending = True) #60 min *24 horas
#print(INCHEUR.columns) #Resetear el index para graficar con dtime 
INCHEUR=INCHEUR.reset_index() 
print(INCHEUR)

#SEGUNDA MONEDA
AAVEEUR, last2 = k.get_ohlc_data("AAVEEUR",interval=1440, ascending = True)
AAVEEUR=AAVEEUR.reset_index() 
print(AAVEEUR)


         dtime        time   open   high    low  close   vwap         volume  \
0   2021-08-10  1628553600  2.364  3.074  2.285  2.389  2.375   87725.416529   
1   2021-08-11  1628640000  2.390  2.982  2.389  2.500  2.578  154972.790530   
2   2021-08-12  1628726400  2.547  2.605  2.305  2.389  2.412   65269.812981   
3   2021-08-13  1628812800  2.405  2.612  2.374  2.609  2.528   40144.334104   
4   2021-08-14  1628899200  2.606  2.627  2.480  2.585  2.584   41095.814459   
..         ...         ...    ...    ...    ...    ...    ...            ...   
502 2022-12-25  1671926400  0.376  0.377  0.370  0.375  0.374    5516.882449   
503 2022-12-26  1672012800  0.374  0.382  0.374  0.381  0.378    1383.307025   
504 2022-12-27  1672099200  0.376  0.381  0.372  0.375  0.378    7271.281019   
505 2022-12-28  1672185600  0.376  0.376  0.360  0.361  0.363   12053.899275   
506 2022-12-29  1672272000  0.363  0.363  0.354  0.356  0.354    6571.719673   

     count  
0      827  
1     1380  


### Resultado Final



---

Una vez que supe bien como funcionaba el OHLC request de Kraken, optimicé el código metiéndolo en una función para así permitir que el usuario indique cuál es la moneda de la cuál quiere descargar su información. 

En esta primera parte decidí que podía mezclar las dos formas que encontré para conectarme a Kraken API. Ya que para descargar el historial de las transacciones usaremos la otra técnica, para rescatar solo los nombres de los pares usaré la librería requests y con un bucle for, separo los pares que sean con euros y los que sean con dólares.  

Si en algún momento para llenar la información de Input del usuario, requieren revisar los nombres de los pares disponibles, pueden llamar a euro_pairs o usd_pairs para obtener la lista de los nombres. 

In [ ]:
resp = requests.get('https://api.kraken.com/0/public/AssetPairs')
resp = resp.json()
euro_pairs = []
for pair in resp['result']:
    if pair.endswith('EUR'):
        euro_pairs.append(pair)
 
euro_pairs

In [ ]:
usd_pairs = []
for pair in resp['result']:
    if pair.endswith('USD'):
        usd_pairs.append(pair)
 
usd_pairs

In [8]:
def get_ohlc(pair: str):
  api = krakenex.API()
  k = KrakenAPI(api)
  df_ohlc_data, last_time = k.get_ohlc_data(pair, interval=1440, ascending = True)
  df_ohlc_data=df_ohlc_data.reset_index() 
  return df_ohlc_data

#Comprobar que se caragron bien los datos 
pair = input("Inserte el par: ")
get_ohlc(pair)

#MINAUSD

Inserte el par: MINAUSD


,dtime,time,open,high,low,close,vwap,volume,count
0,2021-06-01,1622505600,10.000,19.000,2.730,3.250,4.085,3.105506e+06,17735
1,2021-06-02,1622592000,3.230,4.370,2.830,4.080,3.450,1.651540e+06,6048
2,2021-06-03,1622678400,4.080,5.460,3.700,4.550,4.698,2.146178e+06,14915
3,2021-06-04,1622764800,4.560,4.560,3.550,3.850,3.958,1.476516e+06,6119
4,2021-06-05,1622851200,3.860,4.040,3.360,3.470,3.727,1.012177e+06,2965
...,...,...,...,...,...,...,...,...,...
572,2022-12-25,1671926400,0.472,0.472,0.463,0.466,0.468,1.900751e+04,93
573,2022-12-26,1672012800,0.467,0.479,0.464,0.479,0.468,3.635459e+04,149
574,2022-12-27,1672099200,0.480,0.482,0.470,0.473,0.473,1.680010e+05,370
575,2022-12-28,1672185600,0.476,0.476,0.449,0.449,0.458,8.890464e+04,278


## Gráfica de cotizaciones


---
Para la siguiente pregunta trabajaré igualmente creando una función que representa un OHLC candle.  


### Candlestick OHLC 

Luego de haber investigado más acerca de los gráficos más empleados en trading, decidí que era mejor representar un Candlestick con los valores OHLC. Adicionalmente, se incluyó un gráfico de barras para representar los niveles de volumen de dicho par en ese periodo. 

De igual manera, empleamos la función Input() para que el usuario pueda descargar la información de la moneda que desea, y su gráfica. En caso de que sea la misma moneda que la escogida arriba, se puede comentar la línea de input y se recataría el valor dado anteriormente.

In [9]:
def plot_candlestick(pair):
  #Cargar DataFrame de la modena seleccionada 
  df_ohlc_data = get_ohlc(pair)
  #Crear el gráfico de velas 
  candlestick= go.Candlestick(
          x=df_ohlc_data['dtime'],
          open=df_ohlc_data['open'],
          high=df_ohlc_data['high'],
          low=df_ohlc_data['low'],
          close=df_ohlc_data['close'],
          showlegend=False
  )
  volume_bars = go.Bar(
          x=df_ohlc_data['dtime'],
          y=df_ohlc_data['volume'],
          showlegend=False,
          marker={
              "color": "rgba(128,128,128,0.5)",
    }
  )
  fig = go.Figure(candlestick)
  fig = make_subplots(specs=[[{"secondary_y": True}]])
  fig.add_trace(candlestick, secondary_y=True)
  fig.add_trace(volume_bars, secondary_y=False)
  fig.update_layout(title=f"Cotización {pair}", height=800)
  fig.update_yaxes(title="Price $", secondary_y=True, showgrid=True)
  fig.update_yaxes(title="Volume", secondary_y=False, showgrid=False)
 
  fig.show() 
            
pair = input("Inserte el par que desea graficar: ")

plot_candlestick(pair)

#MINAUSD

Inserte el par que desea graficar: MINAUSD


# Indicadores técnicos 

---



## Media Móvil 



In [10]:
ohlc=get_ohlc(pair)

def media_movil (num, ohlc, column):
  ohlc['media_{}_{}'.format(column, num)]= ohlc[column].rolling(window=int(num), min_periods=1).mean()
  return ohlc


#rolling es un comando de la libreria pandas
#min period es porque escogimos una ventana de un día, pero si ese día no está presente en el dataframe, la media se calcula lo mismo y ningun faltante va a inferir 
#num es la ventana de elementos que queremos considerar para el cálculo de la media
#columna sobre la cual queremos calcular 
#crea una nueva columna en funcion del nombre de la columna y del número de elementos que vamos a tener en cuenta

In [11]:
medias=["50","200"] #calcular la media de 50 y 200 días/velas
for m in medias:
  ohlc=media_movil(m, ohlc, "close") 
ohlc.tail() 

,dtime,time,open,high,low,close,vwap,volume,count,media_close_50,media_close_200
572,2022-12-25,1671926400,0.472,0.472,0.463,0.466,0.468,19007.512311,93,0.54752,0.640735
573,2022-12-26,1672012800,0.467,0.479,0.464,0.479,0.468,36354.587458,149,0.54330,0.639005
574,2022-12-27,1672099200,0.480,0.482,0.470,0.473,0.473,168000.973678,370,0.53832,0.637555
575,2022-12-28,1672185600,0.476,0.476,0.449,0.449,0.458,88904.641492,278,0.53436,0.636200
576,2022-12-29,1672272000,0.450,0.450,0.444,0.445,0.447,98187.807003,451,0.53234,0.635275


### Gráfica 

In [12]:
fig_media_movil= go.Figure()

fig_media_movil.add_trace(go.Scatter(
    x=ohlc['dtime'],
    y=ohlc['media_close_50'],
    line_color='rgb(0,176,246)',
    name='Media Móvil 50',
    showlegend=True))

fig_media_movil.add_trace(go.Scatter(
    x=ohlc['dtime'],
    y=ohlc['media_close_200'],
    line_color='rgb(231,107,243)',
    showlegend=True,
    name='Media Móvil 200'))

fig_media_movil.update_layout(
  title=f"Media Móvil mediano y largo plazo - {pair}", height=800,
  yaxis_title="Price $",
  xaxis_title="Tiempo")
fig_media_movil.show()

## RSI



In [13]:
ohlc['diff']=ohlc.close-ohlc.open
ohlc['sub']=ohlc['diff'][ohlc['diff']>0] 
ohlc['baj']=abs(ohlc['diff'][ohlc['diff']<=0]) 
ohlc.head()

,dtime,time,open,high,low,close,vwap,volume,count,media_close_50,media_close_200,diff,sub,baj
0,2021-06-01,1622505600,10.00,19.00,2.73,3.25,4.085,3.105506e+06,17735,3.2500,3.2500,-6.75,NaN,6.75
1,2021-06-02,1622592000,3.23,4.37,2.83,4.08,3.450,1.651540e+06,6048,3.6650,3.6650,0.85,0.85,NaN
2,2021-06-03,1622678400,4.08,5.46,3.70,4.55,4.698,2.146178e+06,14915,3.9600,3.9600,0.47,0.47,NaN
3,2021-06-04,1622764800,4.56,4.56,3.55,3.85,3.958,1.476516e+06,6119,3.9325,3.9325,-0.71,NaN,0.71
4,2021-06-05,1622851200,3.86,4.04,3.36,3.47,3.727,1.012177e+06,2965,3.8400,3.8400,-0.39,NaN,0.39


In [14]:
#Reemplazar con cero los valores de NaN en 'sub' y 'baj'
ohlc.fillna(value=0, inplace=True) 

#calcular el RSI de 14 slots de tiempo-- días en este caso
num_dias= 14 #Para que pueda seleccionar con cuantos días calculamos el RSI

ohlc['media_sub']=ohlc['sub'].rolling(window=num_dias).mean() #Media Móvil exponencial de 'num_dias' períodos alcistas
ohlc['media_baj']=ohlc['baj'].rolling(window=num_dias).mean() #Media Móvil exponencial de 'num_dias' períodos bajistas
ohlc['RSI']=100-(100/(1+(ohlc.media_sub/ohlc.media_baj))) #cálculo del RSI de 'num_dias' días 
ohlc.dropna(inplace=True)
#elimino las columnas que ya use y no necesitaré mas, para quedarme solo con la información importante
ohlc.drop(columns=['diff','sub','baj','media_sub','media_baj'], inplace= True) 
ohlc.tail()

,dtime,time,open,high,low,close,vwap,volume,count,media_close_50,media_close_200,RSI
572,2022-12-25,1671926400,0.472,0.472,0.463,0.466,0.468,19007.512311,93,0.54752,0.640735,19.727891
573,2022-12-26,1672012800,0.467,0.479,0.464,0.479,0.468,36354.587458,149,0.54330,0.639005,25.949367
574,2022-12-27,1672099200,0.480,0.482,0.470,0.473,0.473,168000.973678,370,0.53832,0.637555,24.848485
575,2022-12-28,1672185600,0.476,0.476,0.449,0.449,0.458,88904.641492,278,0.53436,0.636200,20.526316
576,2022-12-29,1672272000,0.450,0.450,0.444,0.445,0.447,98187.807003,451,0.53234,0.635275,21.666667


### Gráfica

In [15]:
ohlc['Nivel 70']= 70
ohlc['Nivel 30']= 30

In [16]:
fig_RSI = go.Figure()

fig_RSI.add_trace(go.Scatter(
    x=ohlc['dtime'],
    y=ohlc['RSI'],
    showlegend=True,
    name='RSI'))

fig_RSI.add_trace(go.Scatter(
    x=ohlc['dtime'],
    y=ohlc['Nivel 70'],
    line_color='rgb(231,107,243)',
    showlegend=True,
    name='Nivel 70'))

fig_RSI.add_trace(go.Scatter(
    x=ohlc['dtime'],
    y=ohlc['Nivel 30'],
    line_color='rgb(0,176,246)',
    name='Nivel 30',
    showlegend=True))

fig_RSI.update_layout(
  title=f"RSI con {num_dias} días- {pair}", height=800,
  yaxis_title="Price $",
  xaxis_title="Tiempo")
fig_RSI.show()

## Visualización cotización y media móvil

In [17]:

fig_final= go.Figure(data=[
    go.Candlestick(
        x=ohlc['dtime'],
        open=ohlc['open'],
        high=ohlc['high'],
        low=ohlc['low'],
        close=ohlc['close'],
        name='OHLC')])

fig_final.add_trace(go.Scatter(
  x=ohlc['dtime'],
  y=ohlc['media_close_50'],
  name='Media Móvil 50',
  showlegend=True))
  
fig_final.add_trace(go.Scatter(
  x=ohlc['dtime'],
  y=ohlc['media_close_200'],
  showlegend=True,
  name='Media Móvil 200'))

fig_final.update_layout(
  title=f"Cotización {pair} y Medias Móviles",
  yaxis_title="Price $",
  xaxis_title= "Filtra según el periodo que te interesa")

fig_final.show()             
